In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from imblearn.combine import SMOTETomek,SMOTEENN
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef
from sklearn.model_selection import StratifiedKFold
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns


from tqdm import tqdm
from typing import Optional


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.multiprocessing import set_start_method

import pytorch_lightning as pl

from torch.utils.data import DataLoader, Dataset
import os


In [2]:
df = pd.read_csv(r'D:\Uni Docs\DSC4996\Dynamic_fraud_detection_system\Data\pre_processed_df.csv')

In [3]:
fraud_trans = df[df['Class'] == 1]

In [4]:
fraud_trans

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
534,406.0,-2.312227,1.951992,-1.609851,3.997906,-0.522188,-1.426545,-2.537387,1.391657,-2.770089,...,0.517232,-0.035049,-0.465211,0.320198,0.044519,0.177840,0.261145,-0.143276,0.00,1
616,472.0,-3.043541,-3.157307,1.088463,2.288644,1.359805,-1.064823,0.325574,-0.067794,-0.270953,...,0.661696,0.435477,1.375966,-0.293803,0.279798,-0.145362,-0.252773,0.035764,529.00,1
4886,4462.0,-2.303350,1.759247,-0.359745,2.330243,-0.821628,-0.075788,0.562320,-0.399147,-0.238253,...,-0.294166,-0.932391,0.172726,-0.087330,-0.156114,-0.542628,0.039566,-0.153029,239.93,1
6072,6986.0,-4.397974,1.358367,-2.592844,2.679787,-1.128131,-1.706536,-3.496197,-0.248778,-0.247768,...,0.573574,0.176968,-0.436207,-0.053502,0.252405,-0.657488,-0.827136,0.849573,59.00,1
6293,7519.0,1.234235,3.019740,-4.304597,4.732795,3.624201,-1.357746,1.713445,-0.496358,-1.282858,...,-0.379068,-0.704181,-0.656805,-1.632653,1.488901,0.566797,-0.010016,0.146793,1.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
278813,169142.0,-1.927883,1.125653,-4.518331,1.749293,-1.566487,-2.010494,-0.882850,0.697211,-2.064945,...,0.778584,-0.319189,0.639419,-0.294885,0.537503,0.788395,0.292680,0.147968,390.00,1
279090,169347.0,1.378559,1.289381,-5.004247,1.411850,0.442581,-1.326536,-1.413170,0.248525,-1.127396,...,0.370612,0.028234,-0.145640,-0.081049,0.521875,0.739467,0.389152,0.186637,0.76,1
279096,169351.0,-0.676143,1.126366,-2.213700,0.468308,-1.120541,-0.003346,-2.234739,1.210158,-0.652250,...,0.751826,0.834108,0.190944,0.032070,-0.739695,0.471111,0.385107,0.194361,77.89,1
280081,169966.0,-3.113832,0.585864,-5.399730,1.817092,-0.840618,-2.943548,-2.208002,1.058733,-1.632333,...,0.583276,-0.269209,-0.456108,-0.183659,-0.328168,0.606116,0.884876,-0.253700,245.00,1


In [5]:
genuine_trans = df[df['Class'] == 0].sample(len(fraud_trans))

In [6]:
df_balanced = pd.concat([fraud_trans,genuine_trans])

In [7]:
features = df_balanced.drop('Class',axis=1)
target = df_balanced['Class']

In [8]:
scaler = StandardScaler()
features[['Time','Amount']] = scaler.fit_transform(features[['Time','Amount']])

In [8]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.2, random_state=1234)

## Model Training - Data Under Sampling

### Logistic Regression

In [9]:
lgt = LogisticRegression()
lgt.fit(X_train, y_train)

LogisticRegression()

In [10]:
y_pred = lgt.predict(X_test)

In [11]:
accuracy_lgt = accuracy_score(y_test, y_pred)
precision_lgt = precision_score(y_test, y_pred)
recall_lgt = recall_score(y_test, y_pred)
f1_score_lgt = f1_score(y_test, y_pred)
mcc_lgt = matthews_corrcoef(y_test, y_pred)

print("Accuracy:", accuracy_lgt)
print("Precision:", precision_lgt)
print("Recall:", recall_lgt)
print("F1-score:", f1_score_lgt)
print("MCC:", mcc_lgt)

Accuracy: 0.9157894736842105
Precision: 0.9545454545454546
Recall: 0.875
F1-score: 0.9130434782608695
MCC: 0.8346696343861983


### SVM

In [12]:
svm = SVC()
svm.fit(X_train, y_train)

SVC()

In [13]:
y_pred_svm = svm.predict(X_test)

In [14]:
accuracy_svm = accuracy_score(y_test, y_pred_svm)
precision_svm = precision_score(y_test, y_pred_svm)
recall_svm = recall_score(y_test, y_pred_svm)
f1_score_svm = f1_score(y_test, y_pred_svm)
mcc_svm = matthews_corrcoef(y_test, y_pred_svm)

print("Accuracy:", accuracy_svm)
print("Precision:", precision_svm)
print("Recall:", recall_svm)
print("F1-score:", f1_score_svm)
print("MCC:", mcc_svm)

Accuracy: 0.9210526315789473
Precision: 0.9764705882352941
Recall: 0.8645833333333334
F1-score: 0.9171270718232044
MCC: 0.8479710021632604


### CREATE GAN MODEL

In [9]:
random_seed = 123
torch.manual_seed(random_seed)

BATCH_SIZE=1000
AVAIL_GPUS = min(1, torch.cuda.device_count())
NUM_WORKERS=0
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
fraud_trans = fraud_trans.drop('Class',axis=1)

In [11]:
fraud_trans[['Time','Amount']] = scaler.fit_transform(fraud_trans[['Time','Amount']])

In [12]:
class CreditCardData(Dataset):
    def __init__(self, data: pd.DataFrame):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, index):
        row = torch.tensor(self.data.iloc[index]).float()
        return row

In [13]:
class DataModule(pl.LightningDataModule):
    def __init__(self, data: pd.DataFrame, batch_size=BATCH_SIZE, num_workers=NUM_WORKERS):
        super().__init__()
        self.data = data
        self.batch_size = batch_size
        self.num_workers = num_workers
        
    def prepare_data(self):
        """
        Empty prepare_data method left in intentionally. 
        https://pytorch-lightning.readthedocs.io/en/latest/data/datamodule.html#prepare-data
        """
        pass
    
    def setup(self, stage: Optional[str] = None):
        train_df, test_df = train_test_split(self.data, random_state=123, test_size=0.2)
        data_mean = train_df.mean()
        data_std = train_df.std()
        train_norm = (train_df - data_mean)/data_std
        test_norm = (test_df - data_mean)/data_std
        self.train_df = train_norm
        self.test_df = test_norm
    
    def train_dataloader(self):
        return DataLoader(dataset=CreditCardData(self.train_df), batch_size=self.batch_size, num_workers=self.num_workers)
    
    def valid_dataloader(self):
        return DataLoader(CreditCardData(self.val_df), batch_size=self.batch_size, num_workers=self.num_workers)
    
    def test_dataloader(self):
        return DataLoader(CreditCardData(self.test_df), batch_size=self.batch_size, num_workers=self.num_workers)

In [14]:
class Discriminator(nn.Module):
    def __init__(self):
        super().__init__()
        self.sequential = nn.Sequential(
            nn.Linear(30, 150),
            nn.ReLU(),
            nn.Linear(150, 50),
            nn.ReLU(),
            nn.Linear(50, 25),
            nn.ReLU(),
            nn.Linear(25, 1)
        )
    
    def forward(self, x):
        tensor = torch.sigmoid(self.sequential(x))
        return tensor

In [15]:
class Generator(nn.Module):
    def __init__(self, latent_dim):
        super().__init__()
        self.sequential = nn.Sequential(
            nn.Linear(latent_dim, 100),
            nn.LeakyReLU(0.2),
            nn.Linear(100, 80),
            nn.LeakyReLU(0.2),
            nn.Linear(80, 40),
            nn.LeakyReLU(0.2),
            nn.Linear(40, 30)
        )
        
    def forward(self, x):
        output = self.sequential(x)
        return output

In [16]:
class GAN(pl.LightningModule):
    def __init__(self, latent_dim=100, lr=0.002):
        super().__init__()
        self.automatic_optimization = False
        self.save_hyperparameters()
        
        self.generator = Generator(latent_dim = self.hparams.latent_dim)
        self.discriminator = Discriminator()
        
        self.validation_z = torch.randn(6, self.hparams.latent_dim)
        
        self.automatic_optimization = False

    def forward(self, z):
        return self.generator(z)
    
    
    def adversarial_loss(self, y_hat, y):
        return F.binary_cross_entropy(y_hat, y)
    
    
    def training_step(self, batch ): 
        real_data = batch
        
        opt_1, opt_2 = self.optimizers()
        z = torch.randn(real_data.size(0), self.hparams.latent_dim)

        
    def configure_optimizers(self):
        lr = self.hparams.lr
        opt_g = torch.optim.Adam(self.generator.parameters(), lr )
        opt_d = torch.optim.Adam(self.discriminator.parameters(), lr )
        return [opt_g, opt_d], []


In [17]:
gnn = GAN()

In [18]:
dm = DataModule(fraud_trans)
trainer = pl.Trainer(max_epochs=100)
trainer.fit(gnn, dm)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: d:\Uni Docs\DSC4996\Dynamic_fraud_detection_system\Src\lightning_logs

  | Name          | Type          | Params
------------------------------------------------
0 | generator     | Generator     | 22.7 K
1 | discriminator | Discriminator | 13.5 K
------------------------------------------------
36.2 K    Trainable params
0         Non-trainable params
36.2 K    Total params
0.145     Total estimated model params size (MB)
c:\Users\Ravin\anaconda3\Lib\site-packages\pytorch_lightning\trainer\connectors\data_connector.py:442: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
c:\Users\

Training: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=100` reached.


In [19]:
z = torch.randn(175000, 100)
output = gnn(z)
output

tensor([[-0.1255, -0.0655,  0.1160,  ..., -0.0322, -0.1999, -0.1380],
        [-0.1705, -0.0180,  0.0969,  ..., -0.0784, -0.1806, -0.0667],
        [-0.1942, -0.0403,  0.0915,  ..., -0.0915, -0.2061, -0.1551],
        ...,
        [-0.1414, -0.0012,  0.1088,  ..., -0.0505, -0.2204, -0.0626],
        [-0.1425,  0.0363,  0.1381,  ..., -0.0976, -0.2072, -0.1303],
        [-0.1539, -0.0324,  0.1680,  ..., -0.0687, -0.1842, -0.0673]],
       grad_fn=<AddmmBackward0>)

In [20]:
only_fraud_df =  pd.DataFrame(output.detach().numpy())

In [21]:
only_fraud_df['Class'] = 1

In [22]:
only_fraud_df

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,Class
0,-0.125472,-0.065518,0.116015,-0.078386,-0.070315,-0.122219,0.017378,-0.126982,0.037647,-0.064911,...,-0.093300,-0.030247,0.125862,-0.075998,-0.203211,-0.113421,-0.032202,-0.199925,-0.137983,1
1,-0.170451,-0.018026,0.096852,-0.065626,0.072934,-0.109999,0.105688,-0.139326,-0.024676,-0.038572,...,0.012400,-0.029055,0.068315,-0.035170,-0.226974,-0.050105,-0.078412,-0.180631,-0.066747,1
2,-0.194220,-0.040289,0.091481,-0.017461,-0.019851,-0.126800,0.100452,-0.153539,-0.021744,-0.032416,...,-0.144041,0.039883,0.146865,-0.086697,-0.240962,-0.144000,-0.091510,-0.206072,-0.155127,1
3,-0.093280,-0.046098,0.146738,-0.053210,0.109236,-0.054069,0.056616,-0.196522,-0.041214,-0.055622,...,-0.050068,0.019684,0.018286,0.022157,-0.161324,-0.015622,-0.043951,-0.208535,-0.073233,1
4,-0.150356,-0.012215,0.128281,-0.067569,0.016128,-0.088675,0.093983,-0.178635,0.011020,-0.070727,...,-0.137437,0.000491,0.095065,-0.012209,-0.176238,-0.080777,-0.105282,-0.187900,-0.072092,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
174995,-0.146697,0.000661,0.108352,-0.014537,0.086563,-0.068104,0.024547,-0.137124,-0.050225,-0.040411,...,-0.033816,0.013552,0.039731,-0.039308,-0.159591,-0.068564,-0.000229,-0.207346,-0.057379,1
174996,-0.106352,0.031905,0.157137,-0.107703,0.037686,-0.094337,0.061924,-0.076955,-0.095596,-0.074425,...,-0.058858,-0.046635,0.119674,-0.080931,-0.211604,-0.091140,-0.044451,-0.162417,-0.099701,1
174997,-0.141414,-0.001164,0.108775,-0.037603,0.031697,-0.133319,0.009418,-0.163063,-0.022776,-0.055269,...,-0.067018,0.008169,0.094791,-0.040277,-0.154987,-0.104819,-0.050531,-0.220356,-0.062604,1
174998,-0.142528,0.036335,0.138094,-0.022376,0.049534,-0.113421,0.134432,-0.096624,-0.050748,-0.087329,...,-0.021029,-0.027467,0.106794,-0.042566,-0.206910,-0.094839,-0.097594,-0.207201,-0.130281,1


In [23]:
df[['Time','Amount']] = scaler.fit_transform(df[['Time','Amount']])

In [24]:
column_names = df.columns
only_fraud_df.columns = column_names

In [25]:
df_final = pd.concat([df,only_fraud_df])

In [26]:
df_final = df_final.sample(frac=1, random_state=42)

In [28]:
df_final = df_final.reset_index(drop=True)

In [29]:
df_final

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,-0.146374,-0.015519,0.085356,-0.040542,0.066677,-0.081207,0.087645,-0.117641,-0.073402,-0.057336,...,0.013769,0.002563,0.101975,-0.060232,-0.191511,-0.090968,-0.066698,-0.206421,-0.086421,1
1,-0.179971,0.010620,0.158013,-0.114025,0.077457,-0.087706,0.063625,-0.105901,-0.054817,-0.068706,...,-0.042756,-0.057189,0.073796,-0.024800,-0.230429,-0.011682,0.010791,-0.156533,-0.053275,1
2,1.219835,2.119089,-1.024633,-0.613399,-0.621483,-0.769342,0.432778,-1.214603,0.185715,-0.019076,...,-0.163955,0.156860,0.181343,0.217964,-0.300916,0.709351,-0.015154,-0.055179,-0.325371,0
3,-0.099612,0.048388,0.174961,-0.056897,0.157342,-0.086699,-0.008830,-0.129328,-0.082722,-0.063709,...,-0.048781,0.036051,0.025908,-0.005768,-0.131239,-0.022695,-0.069082,-0.209291,-0.020680,1
4,-0.327164,1.181842,-0.220696,0.650137,-0.166884,-0.770594,-0.379380,-0.480922,0.187963,0.333115,...,-0.057635,-0.225252,0.156492,0.247896,-0.060545,0.908772,-0.067108,-0.005120,-0.344421,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
458721,1.360712,0.149260,0.988698,-0.605170,-0.788264,1.238822,-0.203064,0.861209,0.067349,-0.279204,...,-0.326673,-0.816542,0.008957,-0.010456,-0.377837,0.128193,0.219217,0.068558,-0.349773,0
458722,-0.144409,-0.056656,0.148645,-0.104169,0.046285,-0.127849,-0.003682,-0.101806,-0.062445,-0.060322,...,0.030523,-0.052311,0.109520,-0.045674,-0.214116,-0.088019,-0.053956,-0.185627,-0.087897,1
458723,-0.312295,1.259310,-0.049484,-0.721776,0.071903,1.864771,3.635628,-0.821682,0.929256,0.128685,...,0.070189,0.104264,-0.107993,1.004823,0.674600,-0.275199,0.045933,0.025074,-0.335356,0
458724,-0.132160,1.982903,-0.134427,-1.161183,0.472515,-0.043755,-1.080473,0.306983,-0.373167,0.454594,...,-0.212230,-0.472417,0.248672,0.025584,-0.195097,0.272711,-0.071660,-0.055715,-0.195658,0


### Exported Generated Dataset

In [32]:
df_final.to_csv('D:/Uni Docs/DSC4996/Dynamic_fraud_detection_system/Data/generated_df.csv', index=False)